In [278]:
import pandas as pd
import numpy as np
from tqdm.notebook import trange, tqdm
import jax
import optax
import matplotlib.pyplot as plt
import importlib
from pathlib import Path

# Force reload of project files
import data, train, models, metrics
importlib.reload(data)
from data import DataLoader


# Params
feature_labels = ['gage_Q','Blue','Green','Red','Nir','Swir1','Swir2'] #Must be list, even for 1 value
target_label = 'tss'
time_slice = slice('2008-01-01', None)
split_time =  np.datetime64('2017-01-01')
num_samples = 10000
sequence_length = 90
batch_size = 256
num_epochs = 50
hidden_size = 16
feature_size = len(feature_labels)
output_size = 1

data_dir = Path("../data/Sag")

dataloader = DataLoader(data_dir = data_dir,
                        basins = ['sag_daily_data','sag_daily_data_2'],
                        features = feature_labels,
                        target = target_label,
                        time_slice = time_slice,
                        split_time = split_time,
                        batch_size = batch_size, 
                        sequence_length = sequence_length,
                        train = True,
                        discharge_col = 'gage_Q',
                        calc_dt=True)

for d in dataloader:
    continue

d['xs'].shape

(176, 1)